# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("pset3_inputdata/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
pharma_doj = doj[doj['id'] == '17-1204']
pharma = pharma_doj['contents'].iloc[0]

# check to make sure we have raw string of press release
print(type(pharma))
pharma

<class 'str'>


'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
## restrict string to only words, removing punctuation and digits
pharma_preprocessed = [word for word in word_tokenize(pharma.lower()) if word.isalpha()]
pharma_preprocessed = ' '.join(pharma_preprocessed)

In [5]:
## tokenize words and then use part of speech tagger
#pharma_preprocessed
words = word_tokenize(pharma_preprocessed)
#words
tags = nltk.pos_tag(words)
#tags

In [6]:
# get adjectives
# got which tags to use from this website : https://cs.nyu.edu/~grishman/jet/guide/PennPOS.html
adjectives = [word for word, tag in tags if tag in ['JJ', 'JJR', 'JJS', 'ADJ']]

adj_counts = {}
for adj in adjectives:
    if adj in adj_counts:
        adj_counts[adj] += 1
    else:
        adj_counts[adj] = 1
print(adj_counts)
sorted_adj_counts = sorted(adj_counts.items(), key=lambda item: item[1], reverse=True)
most_common_adjectives = sorted_adj_counts[:5]
most_common_adjectives

df_adjectives = pd.DataFrame(sorted_adj_counts, columns=['adjective', 'count'])
print(df_adjectives)

{'insys': 2, 'nationwide': 4, 'illegal': 2, 'more': 1, 'synthetic': 1, 'last': 1, 'medical': 1, 'general': 1, 'accountable': 2, 'corporate': 3, 'american': 1, 'current': 2, 'rico': 1, 'other': 3, 'former': 8, 'federal': 3, 'later': 1, 'additional': 1, 'several': 1, 'scottsdale': 1, 'seal': 1, 'national': 1, 'regional': 3, 'bryant': 1, 'joseph': 1, 'panama': 1, 'various': 1, 'many': 1, 'powerful': 2, 'narcotic': 1, 'intense': 1, 'large': 1, 'most': 1, 'reluctant': 1, 'prior': 1, 'potent': 2, 'acting': 1, 'united': 2, 'ongoing': 1, 'opioid': 5, 'utilized': 1, 'acceptable': 1, 'harold': 1, 'special': 7, 'addictive': 3, 'better': 1, 'important': 2, 'pharmaceutical': 2, 'responsible': 1, 'corrupt': 1, 'total': 1, 'patient': 2, 'epidemic': 1, 'dangerous': 1, 'human': 1, 'aware': 1, 'alleged': 2, 'fda': 3, 'criminal': 5, 'washington': 1, 'public': 2, 'controlled': 2, 'trustworthy': 1, 'dea': 1, 'regulatory': 1, 's': 2, 'joint': 1, 'prosecute': 1, 'fraudulent': 1, 'investigative': 2, 'northeas

[('former', 8),
 ('special', 7),
 ('opioid', 5),
 ('criminal', 5),
 ('nationwide', 4)]

      adjective  count
0        former      8
1       special      7
2        opioid      5
3      criminal      5
4    nationwide      4
..          ...    ...
97    assistant      1
98       yeager      1
99     innocent      1
100      guilty      1
101  reasonable      1

[102 rows x 2 columns]


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [54]:
## use before preprocessed string to get named entity recognition
spacy_pressrelease = nlp(pharma)
print(type(spacy_pressrelease))
# testing out NERs
for one_tok in spacy_pressrelease.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

<class 'spacy.tokens.doc.Doc'>
Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Ent

In [8]:
## unique entities w/ tag LAW
law_tag = [one_tok.text for one_tok in spacy_pressrelease.ents if one_tok.label_ == 'LAW']
law_tag = set(law_tag) 
law_tag

{'RICO', 'the Controlled Substances Act'}

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

#### RICO entity explanation

Rico stands for the Racketeer Influenced and Corrupt Organizations Act (why it is given the tag 'LAW'), and it might apply to a pharmaceutical kickbacks case alongside mafia cases because RICO  and the definition of racketeering has expanded to include patterns of repeated crime - which could be a number of white-collar crimes like bribery, counterfeiting, theft, embezzlement, fraud, money laundering - in corporate settings which serves as the "corrupt organization", showing how it can apply to a pharmaceutical kickbacks case. 


source: https://en.wikipedia.org/wiki/Racketeer_Influenced_and_Corrupt_Organizations_Act

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## named entities with DATE label and w/ year or years

def extract_years(spacy_res):
    years = []

    # loop through named entities
    for one_tok in spacy_pressrelease.ents:
        if one_tok.label_ == 'DATE':
            # check if 'year' or 'years' is in the named entity
            if re.search(r'.*(year|years).*', one_tok.text):
                years.append(one_tok.text)
                
    return years


possible_sentences = extract_years(pharma)
print(possible_sentences)

['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
## your code here
year_pattern = r'([^.]*?years[^.]*\.)'
sentences_with_years = re.findall(year_pattern, pharma)
for sentence in sentences_with_years:
    print(sentence)

The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.


The maximum length of sentence for the CEO is 20 years, along with a 3-year probation period if convicted after the indictment.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## your code here for subsetting
topics = ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']
doj_subset = doj[doj['topics_clean'].isin(topics)].copy().reset_index()
doj_subset

,index,id,title,contents,date,topics_clean,components_clean
0,77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
1,155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project S

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
## your code here to define function
def sentiment_analysis(press_string):
    tokenized = word_tokenize(press_string)
    tagged = pos_tag(tokenized)
    
    nlp_res = nlp(press_string)
    without_ner = " ".join([ent.text for ent in nlp_res if not ent.ent_type_])
    
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(without_ner)
    
    return sentiment
   
# test  
#print(sentiment_analysis('A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced.  According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army.  Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008.  Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case.  U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice.  Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims.'))

In [13]:
scores = [sentiment_analysis(press_str) for press_str in doj_subset['contents']]

In [46]:
## your code here executing the function
doj_subset['sentiment_scores'] = doj_subset['contents'].apply(sentiment_analysis)
doj_subset[['contents', 'sentiment_scores']]
doj_subset.shape

,contents,sentiment_scores
0,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.","{'neg': 0.18, 'neu': 0.773, 'pos': 0.047, 'compound': -0.992}"
1,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.","{'neg': 0.111, 'neu': 0.823, 'pos': 0.066, 'compound': -0.891}"
2,"An Alabama native was indicted today and charged with multiple crimes involving travel with intent to engage in illicit sexual conduct with minors and child pornography, announced Assistant Attorney General Leslie R. Caldwell of the

(717, 8)

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [15]:
## already added column to doj_subset df, so now create a copy
doj_subset_wscore = doj_subset.copy().reset_index()
doj_subset_wscore['neg_score'] = doj_subset_wscore['sentiment_scores'].apply(lambda x: x['neg'])
doj_subset_wscore = doj_subset_wscore.sort_values('neg_score', ascending = False)

# print id, contents, neg_score column for two most negative press releases
for i in range(3):
    print(doj_subset_wscore[['id', 'contents', 'neg_score']].iloc[i])

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [16]:
## agg and find the mean compound score by topic
doj_subset_wscore['compound_score'] = doj_subset_wscore['sentiment_scores'].apply(lambda x: x['compound'])
compound_scores = doj_subset_wscore.groupby('topics_clean').agg({'compound_score': 'mean'}).reset_index()
compound_scores

,topics_clean,compound_score
0,Civil Rights,-0.115968
1,Hate Crimes,-0.937175
2,Project Safe Childhood,-0.664339


We might see variation in compound scores due to the nature and emotional contexts of each type of crime. For example, hate crimes will have the most hateful, adverse language in their descriptions because of the nature of the crime, explaining the most negative compound score, whereas Civil Rights for example might not have outwardly hostile language, leading to a less negative compound score. 

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [17]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [18]:
## your code defining a text processing function
def preprocess(words):
    words = words.lower()
    
    stop_words = set(stopwords.words('english')).union(custom_doj_stopwords)
    stemmer = SnowballStemmer('english')
    
    tokens = word_tokenize(words)
    processed = [
        stemmer.stem(tok) for tok in tokens
        if tok.isalpha() and tok not in stop_words and len(tok) >= 4
    ]
    
    return ' '.join(processed)

In [19]:
## your code executing the function
doj_subset_wscore['processed_text'] = [preprocess(content) for content in doj_subset_wscore['contents']]

In [20]:
## your code showing the examples
filtered_doj_wscores = doj_subset_wscore[doj_subset_wscore['id'].isin(['16-718', '16-217'])]
filtered_doj_wscores[['id', 'contents', 'processed_text']]

,id,contents,processed_text
632,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shoot

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [21]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [22]:
dtm_doj = create_dtm(list_of_strings=doj_subset_wscore['processed_text'],
                     metadata=doj_subset_wscore[['id', 'compound_score', 'topics_clean']])


dtm_doj

,index,id,compound_score,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zamora,zane,zealand,zealous,zeeman,zero,zobel,zone,zunggeemog,zwengel
0,13,14-248,-0.9959,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,34,13-312,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,632,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22,11-626,-0.9987,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,567,12-778,-0.9960,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,578,16-294,0.8519,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,577,16-471,0.7717,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,324,17-132,0.9864,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,392,16-539,0.9859,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
def get_topwords(dtm, num_words=10):
    word_counts = dtm[dtm.columns[4:]].sum(axis=0).sort_values(ascending=False)
    return word_counts.head(num_words)

# thresholds for top and bottom 5% compound scores
top_5_percent_threshold = dtm_doj['compound_score'].quantile(0.95)
bottom_5_percent_threshold = dtm_doj['compound_score'].quantile(0.05)

# get subsets based on thresholds
top_5_percent_dtm = dtm_doj[dtm_doj['compound_score'] >= top_5_percent_threshold]
bottom_5_percent_dtm = dtm_doj[dtm_doj['compound_score'] <= bottom_5_percent_threshold]

print("top 10 words in the top 5% most positive:")
print(get_topwords(top_5_percent_dtm))
print("top 10 words in the bottom 5% most negative:")
print(get_topwords(bottom_5_percent_dtm))

# for topics
for topic in dtm_doj['topics_clean'].unique():
    topic_dtm = dtm_doj[dtm_doj['topics_clean'] == topic]
    print("top 10 words for topic " + topic + ":")
    print(get_topwords(topic_dtm))


top 10 words in the top 5% most positive:
agreement     181
disabl        108
ensur         108
state         103
enforc         97
settlement     93
student        90
communiti      83
polic          82
school         80
dtype: int64
top 10 words in the bottom 5% most negative:
assault     188
victim      159
crime       157
hate        126
offic       118
defend      109
sentenc     102
charg        97
anderson     93
guilti       91
dtype: int64
top 10 words for topic Hate Crimes:
victim      590
crime       533
prosecut    476
hate        472
defend      459
sentenc     455
charg       452
guilti      430
feder       426
said        424
dtype: int64
top 10 words for topic Civil Rights:
offic        627
hous         620
discrimin    541
enforc       531
disabl       509
said         497
feder        475
violat       470
state        443
general      408
dtype: int64
top 10 words for topic Project Safe Childhood:
child          1018
exploit         698
sexual          570
safe       

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [24]:
# tokenize text
doj_clean = doj_subset_wscore[doj_subset_wscore.processed_text != ""].copy()
tokenized_text = [wordpunct_tokenize(one_text) 
                for one_text in 
                doj_clean.processed_text]
#tokenized_text

In [25]:
## preprocess and estimate topicmod

### create dictionary
text_proc_dict = corpora.Dictionary(tokenized_text)

### filter dictionary- using 2% as bounds
text_proc_dict.filter_extremes(no_below = round(doj_clean.shape[0]*0.02),
                             no_above = round(doj_clean.shape[0]*0.98))

### create corpus from dictionary
corpus_fromdict_proc = [text_proc_dict.doc2bow(one_text) 
                       for one_text in tokenized_text]

# corpus_fromdict_proc

In [52]:
### estimate model
n_topics = 3
ldamod_proc = gensim.models.ldamodel.LdaModel(corpus_fromdict_proc, 
                                              num_topics = n_topics, 
                                              id2word=text_proc_dict, 
                                              passes=6, alpha = 'auto',
                                              per_word_topics = True, 
                                              random_state = 20)

### print topics and words
topics = ldamod_proc.print_topics(num_words = 15)
for topic in topics:
    print(topic)
    print("\n")
    

(0, '0.031*"child" + 0.021*"exploit" + 0.018*"sexual" + 0.015*"safe" + 0.014*"project" + 0.014*"childhood" + 0.013*"pornographi" + 0.013*"crimin" + 0.013*"children" + 0.012*"prosecut" + 0.011*"victim" + 0.011*"sentenc" + 0.010*"ceo" + 0.010*"minor" + 0.009*"abus"')


(1, '0.014*"victim" + 0.013*"charg" + 0.012*"sentenc" + 0.012*"prosecut" + 0.011*"guilti" + 0.011*"crime" + 0.011*"defend" + 0.011*"feder" + 0.010*"said" + 0.009*"indict" + 0.009*"hate" + 0.009*"prison" + 0.008*"year" + 0.008*"investig" + 0.008*"assault"')


(2, '0.013*"hous" + 0.012*"discrimin" + 0.011*"disabl" + 0.010*"enforc" + 0.008*"offic" + 0.008*"agreement" + 0.008*"said" + 0.008*"state" + 0.008*"court" + 0.007*"feder" + 0.007*"alleg" + 0.007*"violat" + 0.007*"requir" + 0.007*"general" + 0.007*"settlement"')




In [27]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
lda_display_proc = gensimvis.prepare(ldamod_proc, corpus_fromdict_proc, text_proc_dict)
pyLDAvis.display(lda_display_proc)

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [28]:
## your code here to get doc-level topic probabilities 
document_topics = [ldamod_proc.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict_proc]
assert len(document_topics) == len(doj_subset_wscore), "the length of the list is not equal to the number of rows in the doj_subset_wscores dataframe"
print(len(document_topics))
print(len(doj_subset_wscore))
#length of both is 717

717
717


In [29]:
import pandas as pd


doj_subset_wscore['topic_1'] = [
    next((t[1] for t in doc if t[0] == 0))  # Extract the probability for topic 1, or 0 if not found
    for doc in document_topics
]
doj_subset_wscore['topic_2'] = [
    next((t[1] for t in doc if t[0] == 1))  # Extract the probability for topic 1, or 0 if not found
    for doc in document_topics
]
doj_subset_wscore['topic_3'] = [
    next((t[1] for t in doc if t[0] == 2))  # Extract the probability for topic 1, or 0 if not found
    for doc in document_topics
]
doj_subset_wscore['top_topic'] = doj_subset_wscore[['topic_1', 'topic_2', 'topic_3']].idxmax(axis=1)

In [30]:
## topic proportions
topic_distribution = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'], normalize='index') * 100
print(topic_distribution)


top_topic                 topic_1     topic_2   topic_3
topics_clean                                           
Civil Rights             0.327869   34.098361  65.57377
Hate Crimes              0.000000  100.000000   0.00000
Project Safe Childhood  99.397590    0.000000   0.60241


In [53]:
filtered_sample = doj_subset_wscore.sample(n=3, random_state=12)  

for index, row in filtered_sample.iterrows():
    print("document ID:", row['id'])
    print("contents:", row['contents'])
    print("actual topic:", row['topics_clean'])
    print("predicted topic:", row['top_topic'])
    print("-------------------------------------------------------------------------")

document ID: 10-308
contents: WASHINGTON – A federal jury in Sacramento, Calif., today convicted, Eric Clawson, 28, of San Francisco, of a federal hate crime for assaulting an African-American man in a Chico, Calif. bar, announced Thomas E. Perez, Assistant Attorney General for the Civil Rights Division, and Benjamin B. Wagner, U.S. Attorney for the Eastern District of California. Clawson was convicted following a 3-day trial. The evidence at trial showed that on the evening of July 6, 2008, Clawson entered Riley’s Bar & Grill in Chico while the victim was sitting inside with a friend. Shortly after entering the bar, Clawson used a racially-derogatory term to object to the victim’s presence. Clawson repeated this slur several times and, a short while later, without any verbal or physical provocation, approached and punched the victim in the face. The assault rendered the victim unconscious and inflicted injuries to his face and mouth. A second defendant and associate of Clawson who was

The main reason that some manual topics map on more cleanly to an estimated topic than other manual topic is due to the consistency or inconsistency of language used when describing the topic. For example, 'hate crime' documents predominantly map to topic_2, which may have more consistent themes and languages associated with that type of crime, like 'discrimination' or 'violence' or frequent types of hate crimes like graffiti or slurs, which helps our LDA model effectively categorize these documents. On the other hand, 'civil rights' documents can sometimes be misclassified and less consistently mapped, which is likely due to the more diverse content and issues that might lie under the category of 'civil rights'. 
 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [32]:
## A
def create_bigram_onedoc(text):
    words = text.split()
    bigrams = ['_'.join(bigram) for bigram in zip(words, words[1:])]
    return ' '.join(bigrams)


doj_subset_wscore['processed_text_bigrams'] = [create_bigram_onedoc(processed_content) for processed_content in doj_subset_wscore['processed_text']]

In [33]:
## B
filtered_doj_wscores = doj_subset_wscore[doj_subset_wscore['id'].isin(['16-217'])]
filtered_doj_wscores[['id', 'processed_text', 'processed_text_bigrams']]

,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_applaud applaud_citi citi_commiss commiss_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_find find

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

In [34]:
dtm_bigram = create_dtm(list_of_strings=doj_subset_wscore['processed_text_bigrams'],
                     metadata=doj_subset_wscore[['id', 'compound_score', 'topics_clean']])



In [35]:
print("shape of initial dtm matrix: " + str(dtm_doj.shape))
print("shape of bigram dtm matrix: " + str(dtm_bigram.shape))

# bigram dtm matrix has more dimensions than the unigram matrix because bigrams have a larger number of possible word pairs combinations in
# comparison to the single words that a unigram matrix would have, which we can see reflected in the dimensions below. 

shape of initial dtm matrix: (717, 6760)
shape of bigram dtm matrix: (717, 71336)


E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [36]:
# your code here
for topic in dtm_bigram['topics_clean'].unique():
    topic_dtm = dtm_bigram[dtm_bigram['topics_clean'] == topic]
    print("top 10 words for topic " + topic + ":")
    print(get_topwords(topic_dtm))

top 10 words for topic Hate Crimes:
hate_crime       368
plead_guilti     275
year_prison      160
special_agent    117
thoma_perez      111
grand_juri       101
perez_general     95
said_thoma        91
unit_state        87
act_general       85
dtype: int64
top 10 words for topic Civil Rights:
fair_hous         227
deputi_general    221
princip_deputi    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        152
nation_origin     141
consent_decre     127
dtype: int64
top 10 words for topic Project Safe Childhood:
safe_childhood       472
project_safe         472
child_pornographi    445
child_exploit        279
sexual_exploit       223
exploit_children     199
plead_guilti         197
child_sexual         175
exploit_obscen       175
obscen_section       174
dtype: int64


# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [39]:
# main way is to find string similarity - encountered Tfidf Vectorizer in other class, found cosine similarity on google: 
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# tfidf vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(doj_subset_wscore['processed_text'])

cosine_sim = cosine_similarity(tfidf_matrix)
# similarity between each pair of documents ... diagonals are 1 because each document is completely similar with itself
cosine_sim

array([[1.        , 0.0403876 , 0.07665223, ..., 0.00468926, 0.00254177,
        0.00776941],
       [0.0403876 , 1.        , 0.0691215 , ..., 0.02000495, 0.01135529,
        0.01562675],
       [0.07665223, 0.0691215 , 1.        , ..., 0.00638435, 0.0102534 ,
        0.0142577 ],
       ...,
       [0.00468926, 0.02000495, 0.00638435, ..., 1.        , 0.10272586,
        0.03867952],
       [0.00254177, 0.01135529, 0.0102534 , ..., 0.10272586, 1.        ,
        0.06697468],
       [0.00776941, 0.01562675, 0.0142577 , ..., 0.03867952, 0.06697468,
        1.        ]])

In [44]:
# fill diagonals with -1 so we don't consider self-pairs as most similar
np.fill_diagonal(cosine_sim, -1)
cosine_sim

array([[-1.        ,  0.0403876 ,  0.07665223, ...,  0.00468926,
         0.00254177,  0.00776941],
       [ 0.0403876 , -1.        ,  0.0691215 , ...,  0.02000495,
         0.01135529,  0.01562675],
       [ 0.07665223,  0.0691215 , -1.        , ...,  0.00638435,
         0.0102534 ,  0.0142577 ],
       ...,
       [ 0.00468926,  0.02000495,  0.00638435, ..., -1.        ,
         0.10272586,  0.03867952],
       [ 0.00254177,  0.01135529,  0.0102534 , ...,  0.10272586,
        -1.        ,  0.06697468],
       [ 0.00776941,  0.01562675,  0.0142577 , ...,  0.03867952,
         0.06697468, -1.        ]])

In [70]:
max1, max2 = -1, -1  # start with really low similarity so easy to find higher ones
max_pair1 = [None, None]
max_pair2 = [None, None] 


for i in range(len(cosine_sim)):
    for j in range(i + 1, len(cosine_sim[i])): # loop through rest of documents that aren't repeats and haven't been searched yet
        if cosine_sim[i][j] >= max1:
            # add new pair as max1 and place max1 as max2 - because max2 is the samller of the two
            max1, max2 = cosine_sim[i][j], max1
            max_pair1, max_pair2 = [i, j], max_pair1
        elif cosine_sim[i][j] >= max2 and [i, j] != max_pair1 and [j, i] != max_pair1:
            max2 = cosine_sim[i][j]
            max_pair2 = [i, j]

            
print("Top pair 1:")
print("Documents are #" + str(doj_subset_wscore.iloc[max_pair1[0]]['id']) + " and #" + str(doj_subset_wscore.iloc[max_pair1[1]]['id']) + " with similarity of " + str(max1))
print("Top pair 2:")
print("Documents are #" + str(doj_subset_wscore.iloc[max_pair2[0]]['id']) + " and #" + str(doj_subset_wscore.iloc[max_pair2[1]]['id']) + " with similarity of " + str(max2))


Top pair 1:
Documents are #17-546 and #17-231 with similarity of 0.9874669146558988
Top pair 2:
Documents are #17-1089 and #18-705 with similarity of 0.9709252814858463
